# Lossy compression to prevent evasion and poisoning

[Slides](https://danjacobellis.github.io/FTML/compress.slides.html)

<script>
    document.querySelector('head').innerHTML += '<style>.slides { zoom: 1.75 !important; }</style>';
</script>

<center> <h1>
Lossy compression to prevent evasion and poisoning
</h1> </center>

## Types of data threats

* Evasion
  * Exploit knowledge a model that's already been trained
  * Example: email spam filter
    * Attacker wants to avoid detection while preserving the semantic an email
    * Full or partial knowledge of model can be used to find "magic words" that cause an email to be classified as not spam
* Poisoning
  * Attacker contaminates dataset causing major changes to behavior of model
  * Example: facial recognition
    * Attacker wants to prevent facial recognition from working on one or more subjects
    * Attacker uploads altered image to public where dataset is sourced for training

## Gradient attacks

## Lossy compression

## Lossy compression to prevent evasion

## Results

## Compressed training to prevent poisoning

## Results